# Milestone 4 Group 18

## DSCI 525 - Web and Cloud Computing

##### **Authors: Brhuat Musunuru, Jiacheng Wang, Zhenrui Yu and Gurdeepak Sidhu**

## Deploying a Machine Learning Model through Flask API

***Milestone 4:*** In this milestone, you will deploy the machine learning model you trained in milestone 3.

Milestone 4 checklist :

- [X] Use an EC2 instance.
- [X] Develop your API here in this notebook.
- [X] Copy it to ```app.py``` file in EC2 instance.
- [X] Run your API for other consumers and test among your colleagues.
- [X] Summarize your journey.

## Library Imports

In [3]:
## Import all the packages that you need
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

## 1. Develop your API

rubric={mechanics:45}

You probably got how to set up primary URL endpoints from the ```sampleproject.ipynb notebook``` and have them process and return some data. Here we are going to create a new endpoint that accepts a POST request of the features required to run the machine learning model that you trained and saved in last milestone (i.e., a user will post the predictions of the 25 climate model rainfall predictions, i.e., features,  needed to predict with your machine learning model). Your code should then process this data, use your model to make a prediction, and return that prediction to the user. To get you started with all this, I've given you a template which you should fill out to set up this functionality:

***NOTE:*** You won't be able to test the flask module (or the API you make here) unless you go through steps in ```2. Deploy your API```. However, here you can make sure that you develop all your functions and inputs properly.

```python
from flask import Flask, request, jsonify
import numpy as np
import joblib

app = Flask(__name__)

# 1. Load your model here
model = joblib.load("model.joblib")

# 2. Define the model prediction function
def return_prediction(input_data):
    
    data = np.array(input_data).reshape(1,-1) # need two input an array to the model predict method, reshape to correct shape

    return (model.predict(data)[0], model.n_estimators, model.max_depth) #return prediction score, max_depth and n_estimators hyperparamter of the model

# 3. Set up home page using basic html
@app.route("/")
def index():
    # Simple homepage for the Flask API that hosts the model deployed
    return """
    <h1>Welcome to Group 18 rainfall prediction service</h1>
    To use this service, make a JSON post request to the /predict url with 25 climate model outputs.
    """

# 4. Accept POST requests and return model predictions, and model hyperparameters.
@app.route('/predict', methods=['POST'])
def rainfall_prediction():
    content = request.json  # Get's the JSON content of the POST request
    prediction = return_prediction(content["data"]) # only interested with the data field of the JSON POST request
    results = {"Prediction" : prediction[0],        # output the model predictiom
               "Model" : f"Random Forest Regressor", # Name of the regression model used for training
               "Number of estimators" : prediction[1], # Number of estimators used in Random Forest Regressor
               "Max Depth" : prediction[2]}            # Max Depth hyerparameter.
    return jsonify(results)
```

## 2. Deploy your API

rubric={mechanics:40}

Once your API (app.py) is working we're ready to deploy it! For this, do the following:

1. SSH into your EC2 instance from milestone2. There are no issues if you want to spin another EC2 instance; if you plan to do so, make sure you terminate any other running instances.
2. Make a file `app.py` file in your instance and copy what you developed above in there. 

    2.1 You can use the linux editor using ```vi```. More details on vi Editor [here](https://www.guru99.com/the-vi-editor.html). I do recommend doing it this way and knowing some basics like ```:wq,:q!,dd``` will help.
    
    2.2 Or else you can make a file in your laptop called app.py and copy it over to your EC2 instance using ```scp```. Eg: ```scp -r -i "ggeorgeAD.pem" ~/Desktop/worker.py  ubuntu@ec2-xxx.ca-central-1.compute.amazonaws.com:~/```

3. Download your model from s3 to your EC2 instance.
4. Presumably you already have `pip` or `conda` installed on your instance from your previous milestone. You should use one of those package managers to install the dependencies of your API, like `flask`, `joblib`, `sklearn`, etc.

    4.1. You have installed it in your TLJH using [Installing pip packages](https://tljh.jupyter.org/en/latest/howto/env/user-environment.html#installing-pip-packages). if you want to make it available to users outside of jupyterHub (which you want to in this case as we are logging into EC2 instance as user ```ubuntu``` by giving ```ssh -i privatekey ubuntu@<host_name>```) you can follow these [instructions](https://tljh.jupyter.org/en/latest/howto/env/user-environment.html#accessing-user-environment-outside-jupyterhub).
    
    4.2. Alternatively you can install the required packages inside your terminal.
        - Install conda:
            wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
            bash Miniconda3-latest-Linux-x86_64.sh
        - Install packages (there might be others): 
            conda install flask scikit-learn joblib

5. Now you're ready to start your service, go ahead and run `flask run --host=0.0.0.0 --port=8080`. This will make your service available at your EC2 instance's IP address on port 8080. Please make sure that you run this from where ```app.py``` and ```model.joblib``` resides.
6. You can now access your service by typing your EC2 instances public IPv4 address appened with `:8080` into a browswer, so something like `http://<your_EC2_ip>:8080`.
7. You should use `curl` to send a post request to your service to make sure it's working as expected.
>EG: curl -X POST http://your_EC2_ip:8080/predict -d '{"data":[1,2,3,4,53,11,22,37,41,53,11,24,31,44,53,11,22,35,42,53,12,23,31,42,53]}' -H "Content-Type: application/json"

8. Now, what happens if you exit your connection with the EC2 instance? Can you still reach your service?
9. There are several options we could use to help us persist our server even after we exit our shell session. We'll be using `screen`. `screen` will allow us to create a separate session within which we can run `flask` and which won't shut down when we exit the main shell session. Read [this](https://linuxize.com/post/how-to-use-linux-screen/) to learn more on ```screen```.
10. Now, create a new `screen` session (think of this as a new, separate shell), using: `screen -S myapi`. If you want to list already created sessions do ```screen -list```. If you want to get into an existing ```screen -x myapi```.
11. Within that session, start up your flask app. You can then exit the session by pressing `Ctrl + A then press D`. Here you are detaching the session, once you log back into EC2 instance you can attach it using ```screen -x myapi```.
12. Feel free to exit your connection with the EC2 instance now and try accessing your service again with `curl`. You should find that the service has now persisted!
13. ***CONGRATULATIONS!!!*** You have successfully got to the end of our milestones. Move to Task 3 and submit it.

## 3. Summarize your journey from Milestone 1 to Milestone 4
rubric={mechanics:10}
>There is no format or structure on how you write this. (also, no minimum number of words).  It's your choice on how well you describe it.

The group will summarize the journey through DSCI 525 - Web and Cloud Computing based on the Milestone tasks, goals, and learning outcomes.

#### Milestone 1
- Learned how to tackle big data using our laptop, this was quite an annoying task. Since most of the processes we carried out to deal with big data were quite slow and often there was an issue of running out of memory.
- The group also appreciates the fact that we learned how to use the figshare API to download the data of interest. Also learning how to extract zip files in python and combining different CSV files into a single CSV file programmatically.
- Introduction of the concept of DASK which supports parallel computing as an alternative to using Pandas method.
- Learned how to use Pandas and Dask to combine the data files.
- Another helpful insight gained from milestone 1 is how to reduce memory usage in loading data with Pandas by selecting `dtypes`m loading specific columns, converting data types, and just loading in chunks.
- Learned different formats for loading and storing data such as Parquet, Feather, and Arrow.
- These new file formats learned will help deal with Big Data rather than using the generic CSV files which are not conducive for big data.

#### Milestone 2

After experimenting with Big Data using the resources of our laptops, Milestone 2 was designed to transfer Big Data to an instance in the cloud, and use computing resources in the cloud to preprocess and prepare the data for training a machine learning model. Milestone 2 involves the following specific tasks.

- Setting up an Amazon EC2 instance with TLJH, and spinning and connecting to cluster instantiated
- After instantiating the instance the Group learned how to set the JupyterHub within the EC2 instance and add admin users that can access the files in the instance.
- The group also learned how to set up AWS CLI, with AWS configure that makes it convenient to work and access other Amazon web services.
- Also learned how to install packages with pip for JupyterHub on the instance.
- It was good to know how to use Linux commands and how to be a superuser with sudo in the terminal.
- Able to master how to set up an AWS s3 bucket, and how to upload and copy the file from this bucket with AWS CLI using the EC2 instance terminal.
- Wrangle the data on JupyterHub using the resources of the instance, this was cool since the group struggled with this using our laptops for milestone 1.
- They also learned how to push to s3 the wrangled rainfall data that only includes information for Sydney Australia.

#### Milestone 3

After wrangling the rainfall data from milestone 2, then we used this data to train a machine learning model with Random Forest Regression on an EC2 instance. For the first part, we only considered the default hyperparameters to train the model. We checked model accuracy for testing and training sets with the RMSE metric. After that, we set up an EMR cluster with spark and jupyterhub. So that we can use MLib to do cross-validation and tune the hyperparameters for the Random Forest Regression Model. Then we use the optimized hyperparameter to train the Random Forest Regression model and use this model to compute train and test RMSE.

- Learned how to set up an EMR instance with only one node which is the Master node.
- Connecting to JupyterHub with the master node of the EMR cluster
- Learned how to install packages on the terminal of an EMR cluster
- Used an EC2 instance to train a machine learning model on a huge dataset
- Obtained the best hyperparameter for the model with Spark on EMR cluster, which uses the idea of parallel computing, map-reduce, and RDD
- Used DataFrame API to interface with Spark
- Then moved the model trained with optimized hyperparameter to s3 using the joblib package

#### Milestone 4

After training the machine learning model on the rainfall data from Milestone 3. In milestone 4 the group learned how to deploy the trained model to be used by other consumers to predict the rainfall amount based on 25 input features. Learned how to deploy a model with Flask API, and how to send POST requests to the server. Finally, was able to set up the flask API that can be accessed using the curl method.

- Develop the API on the app.py, which contains the flask container and webpages for the webpage that is used to interact with Flask
- Used an EC2 to configure and run Flask
- Also learned how to access the server even after shutting down the Ec2 instance by using the concept of a screen in which the flask server is running until it is detached
- Successfully submitting a POST request to the server and getting back the rainfall prediction based on input data.


## 4. Submission instructions
rubric={mechanics:5}

In the textbox provided on Canvas please put a link where TAs can find the following-
- [ ] This notebook with solution to ```1 & 3```
- [ ] Screenshot from 
    - [ ] Output after trying curl. Here is a [sample](https://github.ubc.ca/MDS-2020-21/DSCI_525_web-cloud-comp_students/blob/master/Milestones/milestone4/images/curl_deploy_sample.png). This is just an example; your input/output doesn't have to look like this, you can design the way you like. But at a minimum, it should show your prediction value.